In [1]:
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
import fiona
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Background

- The code in this notebook was used to come up with a method ("land area overlap") to distribute the ACS data on limited English proficient (LEP) speakers at the Census Tract level to the Election Districts level in NYC. The land area overlap method calculates how much one geographical area for which we have data shares boundaries with another geographical area.
- The information at the Election Districts can be used to understand what poll sites (made up of a combination of Election Districts) will benefit the most from providing language interpreters.
- While this method does not take into account the population density of an area in proportioning data, it is still a highly useful and relevant method, especially when the data on population density does not exist or if the geographical areas follow the same population density distribution.

### 1. Import Election Districts Data and Census Tracts Data (as of Decennial Census 2010) from the NYC Open Data Portal.

[Election Districts](https://data.cityofnewyork.us/City-Government/Election-Districts/h2n3-98hq): The geographical area used by the NY State Board of Elections to determine placement of poll sites. There are 5,840 election districts that roll up to over 1,000 poll sites in Primary and General election periods.

[Census Tracts](https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku): The census tracts are small, permanent geographical unit of a county or its geographical equivalent. According to the Census Bureau, the number of people that reside in a census tract is between 1,200 and 8,000. There are 2,175 census tracts in NYC. 

Analyzing data for smaller geographies can allow for better analysis and census tracts tend to be great in that sense. However, their results should be interpreted with caution when using ACS estimates due to its small sample size.

Variables of interest:
- boro_code: A code associated with each of the five boroughs. (Manhattan = 1, Bronx=2, Brooklyn=3, Queens=4, Staten Island=5)
- boro_ct2010: a unique identifier for each census tract number given that a census tract number may not be unique.
- boro_name: name of one of the five boroughs of NYC
- ct2010: census tract number
- ctlabel: truncated census tract number

In [2]:
census_tracts_2010_gpd.head()

NameError: name 'census_tracts_2010_gpd' is not defined

### 1.1. Import Election Districts Shapefile (clipped to the shore) from NYC Open Data.  *Data Source: Decennial Census 2010*

In [ ]:
def import_shapefile_and_area_in_sqkm(four_by_four_code):
    '''
    Import a shapefile and calculate the area in square kilometer for each tile. 
    
    Input:
        - four_by_four_code: a string that contains 8 alphanumeric letters assigned 
           to each open dataset on NYC Open Data Portal.
    
    Returns:
        - a geopandas dataframe.
    
    '''
    
    # import the shapefile as GeoJSON from the open data portal
    request = requests.get("https://data.cityofnewyork.us/api/geospatial/" + \
                           four_by_four_code + "?method=export&format=GeoJSON")
    
    visz = fiona.ogrext.buffer_to_virtual_file(bytes(request.content))
    
    # read the features from a fiona collection into a GeoDataFrame
    with fiona.Collection(visz, driver='GeoJSON') as f:
        nyc_opendata_gpd = geopandas.GeoDataFrame.from_features(f, crs=f.crs)
    
    print("coordinate reference system of the shapefile: ", nyc_opendata_gpd.crs)
    
    # convert the geometry into an equal projection area, and then meter square into kilometer square

    # source: https://gis.stackexchange.com/questions/218450/getting-polygon-areas-using-geopandas
    nyc_opendata_gpd['area_in_sqkm'] = nyc_opendata_gpd['geometry'].to_crs({'init': 'epsg:3395'})\
                   .map(lambda p: p.area / 10**6)
    
    return nyc_opendata_gpd

In [ ]:
ed_nyc_od_gpd = import_shapefile_and_area_in_sqkm("h2n3-98hq")

In [ ]:
ed_nyc_od_gpd.head()

In [ ]:
# number of unique ED's
ed_nyc_od_gpd['elect_dist'].nunique()

Plot the map.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_aspect('equal')

ed_nyc_od_gpd.plot(ax=ax, color='white', edgecolor='black', linewidth=0.75, alpha=0.8)

plt.title("Election Districts of NYC as of June 2021")

### 1.2. Import Census Tracts Shapefile from NYC Open Data (2,165 unique Census Tract's). *Data Source: Decennial Census 2010*

In [ ]:
census_tracts_2010_gpd = import_shapefile_and_area_in_sqkm("fxpq-c8ku")

In [ ]:
census_tracts_2010_gpd.head()

Plot the map.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 13))
ax.set_aspect('equal')
census_tracts_2010_gpd.plot(ax=ax, color='white', edgecolor='black', linewidth=0.75, alpha=0.8)
plt.title("Census Tracts of NYC - Decennial Census 2010")

In [ ]:
census_tracts_2010_gpd.crs

In [ ]:
census_tracts_2010_gpd.shape

### 2. Overlay both Election Districts and Census Tracts shapefiles

What does the overlay function do?
Argument of 'how'?

In [ ]:
eds_to_tracts_intersection = geopandas.overlay(ed_nyc_od_gpd, census_tracts_2010_gpd, how='intersection')

In [ ]:
eds_to_tracts_intersection.head(5)

In [ ]:
ax = eds_to_tracts_intersection.plot(alpha=0.7, figsize=(13, 13))

ed_nyc_od_gpd.plot(ax=ax, facecolor='none', edgecolor='g')
census_tracts_2010_gpd.plot(ax=ax, facecolor='none', edgecolor='w')

plt.title("Map of Election Districts (green border) and Census Tracts (white border) Overlapped")

If there were any areas that were not filled with blue color, that would indicate that there's no overlapping boundaries.

### 3. After overlaying, calculate the intersection area into square km and identify the census tract-to-election district ratio (i.e. percentage of census tract that shares boundaries with an election district)

In [ ]:
# calculate intersection area into square km
eds_to_tracts_intersection['intersection_sqkm'] = \
                eds_to_tracts_intersection['geometry'].to_crs({'init': 'epsg:3395'})\
               .map(lambda p: p.area / 10**6)

In [ ]:
eds_to_tracts_intersection.head()

Add two metrics to show a) the proportion of a single ED that falls in a census tract and b) the proportion of a single census tract that is occupied by the census tract.

In [ ]:
eds_to_tracts_intersection['elect_dist_area_percent_of_tract_km'] = \
    eds_to_tracts_intersection['intersection_sqkm']/eds_to_tracts_intersection['area_in_sqkm_2']

eds_to_tracts_intersection['elect_dist_area_percent_as_a_whole_km'] = \
    eds_to_tracts_intersection['intersection_sqkm']/eds_to_tracts_intersection['area_in_sqkm_1']

Rearrange the order of the dataframe by borough, census tract and the election district.

In [ ]:
eds_to_tracts_intersection_groupby = eds_to_tracts_intersection.dissolve(
    by=['boro_name', 'boro_ct2010', 'ct2010', 'elect_dist'],
    aggfunc='sum'
    ).reset_index()

In [ ]:
eds_to_tracts_intersection_groupby.shape

In [ ]:
eds_to_tracts_intersection_groupby.head()

Perform a quick QA to check that the proportions of a all ED's sum up to 1 (i.e. 100%). If it does, then it shows that all the ED's of NYC are completely mapped to one or more census tracts

In [ ]:
election_district_proportions_QA = eds_to_tracts_intersection_groupby\
    .groupby(['elect_dist'])['elect_dist_area_percent_as_a_whole_km'].sum().reset_index()\
    .sort_values('elect_dist_area_percent_as_a_whole_km', ascending=True)

In [ ]:
election_district_proportions_QA.head()

In [ ]:
# subtracting 0.01 from the lower bound and adding 0.01 to upper bound to account for rounding issues in "elect_dist_area_percent_as_a_whole_km"
election_district_proportions_QA[(election_district_proportions_QA['elect_dist_area_percent_as_a_whole_km'] < 0.99) | \
                                 (election_district_proportions_QA['elect_dist_area_percent_as_a_whole_km'] > 1.01)]

Given the search results are empty, it shows that all ED's were able to map completely to the census tracts.

### 4. Demo: Plot the intersection areas of sample neighborhood to show how the data can be distributed to a different geographic level

In [ ]:
# Prospect Heights is a neighborhood in central Brooklyn

prospect_heights_ed = ed_nyc_od_gpd[ed_nyc_od_gpd['elect_dist'].isin(['57063', '57071'])]
prospect_heights_census_tracts = census_tracts_2010_gpd[census_tracts_2010_gpd['ntaname'] == 'Prospect Heights']

#intersect the ED's and census tracts of Prospect Heights
prospect_heights_ed_and_tracts_intersection = geopandas.overlay(prospect_heights_ed, \
                                                                prospect_heights_census_tracts, how='intersection')

In [ ]:
ax = prospect_heights_ed_and_tracts_intersection.plot(figsize=(10, 10))

# plot the ED's and census tracts of Prospect Heights
prospect_heights_ed.plot(ax=ax, facecolor='none', edgecolor='r');
# prospect_heights_ed.apply(lambda x: ax.annotate(s=x['elect_dist'], \
#                 xy=x.geometry.centroid.coords[0], ha='center', color='w'),axis=1);

prospect_heights_census_tracts.plot(ax=ax, facecolor='none', edgecolor='k');
prospect_heights_census_tracts.apply(lambda x: ax.annotate(s=x['boro_ct2010'], \
                xy=x.geometry.centroid.coords[0], ha='right'),axis=1);
# Calculate the intersection area of ED's and census tracts in square kilometer
prospect_heights_ed_and_tracts_intersection['intersection_sqkm'] = \
                prospect_heights_ed_and_tracts_intersection['geometry'].to_crs({'init': 'epsg:3395'})\
               .map(lambda p: p.area / 10**6)

# identify the metrics that show proportion of each ED and census tract 
prospect_heights_ed_and_tracts_intersection['elect_dist_area_percent_of_tract_km'] = \
                prospect_heights_ed_and_tracts_intersection['intersection_sqkm']/\
                prospect_heights_ed_and_tracts_intersection['area_in_sqkm_2']

prospect_heights_ed_and_tracts_intersection['elect_dist_area_percent_as_a_whole_km'] = \
                prospect_heights_ed_and_tracts_intersection['intersection_sqkm']/\
                prospect_heights_ed_and_tracts_intersection['area_in_sqkm_1']

#annonate the first metric "proportion of election district that belongs to a census tract" results calculated above on the map
prospect_heights_ed_and_tracts_intersection.apply(\
                lambda x: ax.annotate(s=round(x['elect_dist_area_percent_of_tract_km'], 2),\
                xy=x.geometry.centroid.coords[0], ha='left', color='w'),axis=1);

The visualization above is of only two election districts (the horizontal election district above, 57063, and the one below, 57071) in Prospect Heights. In order to proportion the data from the Census Tract 3016300 to the above horizontal ED, we'll multiply the data by 0.15. 

E.g. If there are 1,500 language speakers who speak Russian in Census Tract 3016300, then the number of Russian speakers in the Election District 57063 will be 225.